# Speeding up code

## Lazy evaluation  

Under the hood nchack relies mostly on CDO to carry out the specified manipulation of netcdf files. Each time CDO is called a new temporary file is generated. This has the potential to result in slower than necessary processing chains, as IO takes up far too much time.

I will demonstrate this using a netcdf file os sea surface temperature. To download the file we can just use wget:

In [1]:
import nchack as nc
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
#! wget ftp://ftp.cdc.noaa.gov/Datasets/COBE2/sst.mon.ltm.1981-2010.nc
clear_output()

We can then set up the dataset which we will use for manipulating the SST climatology.

In [2]:
ff =  "sst.mon.ltm.1981-2010.nc"
sst = nc.open_data(ff)

Now, let's select the variable sst, clip the file to the northern hemisphere, calculate the mean value in each grid cell for the first half of the year, and then calculate the spatial mean.

In [3]:
sst.select_variables("sst")
sst.clip(lat = [0,90])
sst.select_months(list(range(1,7)))
sst.mean()
sst.spatial_mean()

The dataset's history is as follows:

In [4]:
sst.history

['cdo -selname,sst sst.mon.ltm.1981-2010.nc /tmp/nchackxthdlmcunchacktmpb9hnczb7.nc',
 'cdo -sellonlatbox,-180,180,0,90 /tmp/nchackxthdlmcunchacktmpb9hnczb7.nc /tmp/nchackxthdlmcunchacktmpygkddpqe.nc',
 'cdo -selmonth,1,2,3,4,5,6 /tmp/nchackxthdlmcunchacktmpygkddpqe.nc /tmp/nchackxthdlmcunchacktmphilqd_1x.nc',
 'cdo -timmean /tmp/nchackxthdlmcunchacktmphilqd_1x.nc /tmp/nchackxthdlmcunchacktmpzpy8e0zk.nc',
 'cdo -fldmean /tmp/nchackxthdlmcunchacktmpzpy8e0zk.nc /tmp/nchackxthdlmcunchacktmpjz5am1ap.nc']

In total, there are 5 operations, with temporary files created each time. However, we only want to generate one temporary file. So, can we do that? Yes, thanks to CDO's method chaining ability.
If we want to utilize this we need to set the dataset's evaluation to lazy. Once this is done nchack will only evaluate things either when it needs to, e.g. you call a method that cannot possibly be chained, or if you release it.
This works as follows:

In [5]:
ff =  "sst.mon.ltm.1981-2010.nc"
sst = nc.open_data(ff)
sst.lazy()
sst.select_variables("sst")
sst.clip(lat = [0,90])
sst.select_months(list(range(1,7)))
sst.mean()
sst.spatial_mean()
sst.release()

We can now see that the history is much cleaner, with only one command.

In [6]:
sst.history

['cdo -L -fldmean -timmean -selmonth,1,2,3,4,5,6 -sellonlatbox,-180,180,0,90 -selname,sst sst.mon.ltm.1981-2010.nc /tmp/nchackxthdlmcunchacktmps2a0lxos.nc']

How does this impact run time? Let's time the original, unchained method.

In [7]:
%%time
ff =  "sst.mon.ltm.1981-2010.nc"
sst = nc.open_data(ff)
sst.select_variables("sst")
sst.clip(lat = [0,90])
sst.select_months(list(range(1,7)))
sst.mean()
sst.spatial_mean()

CPU times: user 3.95 ms, sys: 64.8 ms, total: 68.8 ms
Wall time: 595 ms


In [8]:
%%time
ff =  "sst.mon.ltm.1981-2010.nc"
sst = nc.open_data(ff)
sst.lazy()
sst.select_variables("sst")
sst.clip(lat = [0,90])
sst.select_months(list(range(1,7)))
sst.mean()
sst.spatial_mean()
sst.release()

CPU times: user 2.64 ms, sys: 4.79 ms, total: 7.43 ms
Wall time: 110 ms


This was almost 4 times faster. Exact speed improvements, will of course depend on specific IO requirements, and some times using lazy evaluation will make negligible impact, but in others can make code over 10 times fasteExact speed improvements, will of course depend on specific IO requirements, and some times using lazy evaluation will make negligible impact, but in others can make code over 10 times faster.

In some cases, we may want lazy evaluation by default on all datasets. In this case we can use options:

In [12]:
nc.options(lazy = True)
ff =  "sst.mon.ltm.1981-2010.nc"
sst = nc.open_data(ff)
sst.select_variables("sst")
sst.clip(lat = [0,90])
sst.select_months(list(range(1,7)))
sst.mean()
sst.spatial_mean()
sst.release()
sst.history

['cdo -L -fldmean -timmean -selmonth,1,2,3,4,5,6 -sellonlatbox,-180,180,0,90 -selname,sst sst.mon.ltm.1981-2010.nc /tmp/nchackxthdlmcunchacktmp7uvbjivd.nc']

## Using thread-safe libraries

TBC